finance-datareader를 설치한다

In [1]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


라이브러리 호출

In [2]:
import FinanceDataReader as fdr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


주식정보 읽기
삼성전자 코드='005930', 2016년 데이터부터 다운로드

주식 종목코드 검색 참조

In [3]:
samsung = fdr.DataReader('005930', '2016')
print(samsung)


             Open   High    Low  Close    Volume    Change
Date                                                      
2016-01-04  25200  25200  24100  24100    306939 -0.043651
2016-01-05  24040  24360  23720  24160    216002  0.002490
2016-01-06  24160  24160  23360  23500    366752 -0.027318
2016-01-07  23320  23660  23020  23260    282388 -0.010213
2016-01-08  23260  23720  23260  23420    257763  0.006879
...           ...    ...    ...    ...       ...       ...
2022-11-09  62000  62200  61300  62000  14045592  0.003236
2022-11-10  61400  61500  60400  60400  21087633 -0.025806
2022-11-11  63100  63200  62300  62900  20037163  0.041391
2022-11-14  62900  62900  61700  61900  15973416 -0.015898
2022-11-15  62200  62500  61600  62400  12207263  0.008078

[1689 rows x 6 columns]


Open => 시가(시작가격)

In [4]:
openValues = samsung[['Open']]


데이터 전처리 => 데이터 정규화 (0~1 사이값으로 변환)

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0, 1))
scaled = scaler.fit_transform(openValues)


학습용과 검증용 데이터 분리

마지막 TEST_SIZE 만큼의 데이터를 검증용 데이터로 사용

In [6]:
TEST_SIZE = 200
train_data = scaled[:-TEST_SIZE]
test_data = scaled[-TEST_SIZE:]


순차훈련 데이터 생성함수

In [7]:
def make_sample(data, window):
    train = []
    target = []
    for i in range(len(data)-window):
        train.append(data[i:i+window])
        target.append(data[i+window])
    return np.array(train), np.array(target)


순차훈련 데이터 생성

In [8]:
X_train, y_train = make_sample(train_data, 30)

LSTM 학습을 위한 Import

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM


LSTM 모델생성

셀의 갯수 = 16

In [10]:
model = Sequential()
model.add(LSTM(16, 
                input_shape=(X_train.shape[1], 1), 
                activation='tanh', 
                return_sequences=False)
          )
model.add(Dense(1))   #최종 출력 Layer

model.compile(optimizer = 'adam', loss = 'mean_squared_error')


학습 실행

In [ ]:
model.fit(X_train, y_train, epochs = 100, batch_size = 16)


Epoch 1/100
92/92 [==============================] - 6s 20ms/step - loss: 0.0394
Epoch 2/100
92/92 [==============================] - 2s 20ms/step - loss: 0.0023
Epoch 3/100
92/92 [==============================] - 2s 20ms/step - loss: 0.0012
Epoch 4/100
92/92 [==============================] - 2s 20ms/step - loss: 0.0011
Epoch 5/100
92/92 [==============================] - 2s 17ms/step - loss: 0.0011
Epoch 6/100
92/92 [==============================] - 2s 18ms/step - loss: 0.0011
Epoch 7/100
92/92 [==============================] - 2s 18ms/step - loss: 0.0011
Epoch 8/100
92/92 [==============================] - 1s 11ms/step - loss: 0.0011
Epoch 9/100
92/92 [==============================] - 1s 11ms/step - loss: 0.0011
Epoch 10/100
92/92 [==============================] - 1s 12ms/step - loss: 0.0011
Epoch 11/100
92/92 [==============================] - 1s 11ms/step - loss: 0.0011
Epoch 12/100
92/92 [==============================] - 1s 11ms/step - loss: 0.0011
Epoch 13/100
92/92 [=====

샘플 Test 데이터 생성 및 예측

In [ ]:
X_test, y_test = make_sample(test_data, 30)
pred = model.predict(X_test)


결과를 그래프로 보기

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 9))
plt.plot(y_test, label='stock price')
plt.plot(pred, label='predicted stock price')
plt.legend()
plt.show()
